<h1> NEAT Implementation Practice </h1>
A first-pass attempt at implementing NEAT by translating SethBling's neatevolve.lua script into Python.

# Some Notes
Here are some things about which knobs you can turn in each cell:
1. Imports. Don't touch anything there unless you wanna try something out at your own risk.
2. Environment Declarations. Change the envName string if you want to test out other environments, but just note that you may need to change some other parameters in the evaluation cell as well if you do so.
3. Hyperparameters. Change as you please, but I would recommend sticking to the values from the NEAT paper and/or neatevolve.lua.
4. Some functions. I wouldn't touch this.
5. Genome Class. I wouldn't touch this, but please feel free to look at the mess I've made. I have recently moved the crossover and delta functions as static methods to this class.
8. Evaluation. As mentioned in 2), if you change the environment, then you made need to change the evaluateAction method of the genome or some of the target fitness requirements for episode termination.

In [1]:
import gym
import numpy as np # might not even need this ?
import random # pseudo-random|
import math
import time # for timing how long it runs for 
import PySimpleGUI as sg
import pickle 

In [2]:
# first define the environment and get the both the action and observation spaces 
envName = 'MsPacman-ram-v0' # just need to change this if you wanna try it in environments
env = gym.make(envName)

# simple gui stuff
sg.theme = ('DarkAmber')
layout = [  [sg.Text('Simulation Info')],
            [sg.Text('Generation: '), sg.Text(0, key='GENER-NUM', size = (20,1))],
            [sg.Text('Genome: '), sg.Text(0, key='GENOM-NUM',size = (20,1))],
            [sg.Text('Species: '), sg.Text(0, key='SPECIES-NUM',size = (20,1))],
            [sg.Text('Fitness: '), sg.Text(0, key='FITNESS-NUM',size = (20,1))]]



# # print out the observation and action spaces
# print(env.action_space.shape[0]) # number of valid actions we can take 
# print(env.observation_space.shape[0]) # number of "sensors" of the environment that we have (number of inputs)

# NOTE: Need to keep track of the valid values these can take 
# NOTE: Also need to have inputs + 1, with the 1 being the bias (stc?)
# NOTE: changes between Box() and Discrete() depending on environment
outputs = env.action_space.n
inputs = env.observation_space.shape[0] + 1


In [3]:
# hyperparameters, according to section 4.1 Parameter Settings (changed some parameters for myself)
deltaThreshold = 1.0

c1 = c2 = 1.0
c3 = 0.4
perturbVal = 0.05


numUntilStagnant = 15


genomeMutationChance = 0.25 # neatevolve.lua has this pretty low 
connectionUniformPerturb = 0.9
connectionRandomValue = 1 - connectionUniformPerturb
disableGeneChance = 0.75 
mutateNoCrossoverChance = 0.25
interspeciesMatingRate = 0.01 
smallPopulationNewNodeChance = 0.05
largePopulationNewNodeChance = 0.05 # using these right now 
smallPopulationNewConnectionChance = 0.03
largePopulationNewConnectionChance = 0.3 # using these right now 

In [4]:
# activation functions 
def sigmoid(x):
    return 1/(1+np.exp(-x))

def steepsigmoid(x):
    return 1/(1+np.exp(-4.9*x))

In [5]:
# genome class 
class Genome:
    # initialize class variables 
    globalInnovationNumber = 1
    def __init__(self, inputs=1, outputs=1):
        self.globalInnovation = 1
        self.nodeGenes = {}
        self.connectGenes = {}
        self.fitness = 0
        self.sharedFitness = 0
        self.species = 0
        self.inputCount = inputs
        self.outputCount = outputs
        self.mutationSuccess = False 
        
    def setGenes(self, nodeGenes, connectGenes):
        self.nodeGenes = nodeGenes.copy()
        self.connectGenes = connectGenes.copy()
        
    @staticmethod
    def copyGenome(genome):
        g = Genome(genome.inputCount, genome.outputCount)
        g.setGenes(genome.nodeGenes, genome.connectGenes)
        return g
    
    @staticmethod
    def resetGlobalInnovationNumber():
        Genome.globalInnovationNumber = 1
        # print('Genome global innovation number reset back to: {}'.format(Genome.globalInnovationNumber))
    
    @staticmethod
    def crossover(genome1, genome2):
        if (genome2.fitness > genome1.fitness):
            temp = genome1
            genome1 = genome2
            genome2 = temp

        g = Genome(genome1.inputCount, genome1.outputCount)
        g.initEmptyGenome()
        g1GenomeList = {}
        for connection in genome1.connectGenes:
            #print(genome1.connectGenes[connection])
            g1GenomeList[genome1.connectGenes[connection][0]]=[connection,genome1.connectGenes[connection][1],genome1.connectGenes[connection][2]]
        g2GenomeList = {}
        for connection in genome2.connectGenes:
            g2GenomeList[genome2.connectGenes[connection][0]]=[connection,genome2.connectGenes[connection][1],genome2.connectGenes[connection][2]] 

        # when crossing over, the genes in both genomes with the same innovation numbers are lined up
        # in composing the offspring, genes are randomly chosen from either parent at matching genes,
        # whereas all excess or disjoint genes are always included from the more fit parent 
        for i in range(max(len(g1GenomeList), len(g2GenomeList))):
            if (i+1 in g1GenomeList and i+1 in g2GenomeList):
                if (random.random() <= 50): # inherit from genome1
                    # print('inherited from parent 1')
                    g.connectGenes[g1GenomeList[i+1][0]] = [i+1, g1GenomeList[i+1][1], g1GenomeList[i+1][2]]
                else: # inherit from genome2 
                    # print('inherited from parent 2')
                    g.connectGenes[g2GenomeList[i+1][0]] = [i+1, g2GenomeList[i+1][1], g2GenomeList[i+1][2]]
                if (g1GenomeList[i+1][2] == False or g1GenomeList[i+1][2] == False):
                    if (random.random() <= disableGeneChance):
                        # print('inherited disabled gene')
                        g.connectGenes[g1GenomeList[i+1][0]][2] = False
                    else:
                        # print('re-enabled inherited gene')
                        g.connectGenes[g1GenomeList[i+1][0]][2] = True
            if (i+1 in g1GenomeList and i+1 not in g2GenomeList):
                g.connectGenes[g1GenomeList[i+1][0]] = [i+1, g1GenomeList[i+1][1], g1GenomeList[i+1][2]]
            # insert the node genes 

        for c in g.connectGenes:
            if c[0] in g1.nodeGenes:
                g.nodeGenes[c[0]] = g1.nodeGenes[c[0]]
            elif c[0] in g2.nodeGenes:
                g.nodeGenes[c[0]] = g2.nodeGenes[c[0]]

            if c[1] in g1.nodeGenes:
                g.nodeGenes[c[1]] = g1.nodeGenes[c[1]]
            elif c[0] in g2.nodeGenes:
                g.nodeGenes[c[1]] = g2.nodeGenes[c[1]]

        g.clearNodeValues()
        return g
    
    @staticmethod
    def delta(genome1, genome2, c1=0.1, c2=0.1, c3=0.1):
        if (not genome1.connectGenes or not genome2.connectGenes):
            return 0
        connectionMismatch = []
        for connection1 in genome1.connectGenes:
            if (connection1 not in genome2.connectGenes):
                connectionMismatch.append(connection1)
        for connection2 in genome2.connectGenes:
            if (connection2 not in genome1.connectGenes):
                connectionMismatch.append(connection2)
        diffsum = 0
        num = 0
        for connection1 in genome1.connectGenes:
            if (connection1 not in connectionMismatch):
                diffsum += abs(genome1.connectGenes[connection1][1] -
                               genome2.connectGenes[connection1][1])
                num += 1
        averageWeightDifference = diffsum / num
        #print(averageWeightDifference)
        N = 0
        if len(genome1.connectGenes) >= len(genome2.connectGenes):
            N = len(genome1.connectGenes)
        else:
            N = len(genome2.connectGenes)
        print(((len(connectionMismatch)*(c1 + c2))/N) + (c3*averageWeightDifference))
        return ((len(connectionMismatch) *
                 (c1 + c2)) / N) + (c3 * averageWeightDifference)
    
    def clearGenome(self):
        self.globalInnovation = 1
        self.nodeGenes.clear()
        self.connectGenes.clear()
        
    def clearNodeValues(self):
        for node in self.nodeGenes:
            self.nodeGenes[node][1] = 0.0
            
    def initEmptyGenome(self):
        self.clearGenome()
        for i in range(self.inputCount):
            self.insertNode('input')
        for i in range(self.outputCount):
            self.insertNode('output')
            
    def resetFitness(self):
        self.fitness = 0
        self.sharedFitness = 0
        
    def initRandomGenome(self):
        self.initEmptyGenome()
        
        # brute force connect each new input node to each output node 
        for node in self.nodeGenes:
            if self.nodeGenes[node][0] == 'input':
                for otherNode in self.nodeGenes:
                    if self.nodeGenes[otherNode][0] == 'output':
                        self.insertConnection(outNode=node, inNode=otherNode, weight=random.uniform(-1, 1))
             
    def insertNode(self, nodeType='input', value = 0.0):
        self.nodeGenes[len(self.nodeGenes)+1]=[nodeType, value]
        
    def insertConnection(self, outNode, inNode, weight = 0.5, isExpressed = True):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes): # check if the nodes even exist 
            if ((outNode, inNode) not in self.connectGenes and (inNode, outNode) not in self.connectGenes and self.nodeGenes[inNode][0] != 'input' and outNode != inNode and self.nodeGenes[outNode][0] != 'output'): # check that the connection doesnt already exist
                # also dont allow recurrent connections (only feedforward)
                self.connectGenes[(outNode, inNode)] = [Genome.globalInnovationNumber, weight, isExpressed]
                Genome.globalInnovationNumber += 1
                self.mutationSuccess = True
            else:
                return
                #print('Connection already exists. Did not insert')
        else:
            return
            #print('Could not find node(s). Did not insert.')
            
    def checkConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                return self.connectGenes[(outNode, inNode)][2]
            else:
                print('Could not find the connection')
        else:
            print('Could not find node(s). Did not disable.')
            
    def disableConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.connectGenes[(outNode, inNode)][2] = False
            else:
                print('Could not find connection to disable. Did not disable.')
        else: 
            print('Could not find node(s). Did not disable.')
    
    def mutateAddConnection(self, outNode, inNode):
        self.insertConnection(outNode, inNode, random.uniform(-1, 1), True)
            
    def mutateAddRandomConnection(self):
        timesAttemptedToConnected = 0
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            self.mutateAddConnection(random.choice(list(self.nodeGenes)),random.choice(list(self.nodeGenes)))
            timesAttemptedToConnected +=1
            if (timesAttemptedToConnected > 100):
                print('Attempted to connect too many times. probably already full')
                return
        self.mutationSuccess = False
        
    def mutateAddNode(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.insertNode('hidden', '0.0')
                # disable old connection'
                self.disableConnection(outNode, inNode)
                self.insertConnection(outNode, len(self.nodeGenes), 1.0, True)
                # get original weight 
                originalWeight = self.connectGenes[(outNode, inNode)][1]
                self.insertConnection(len(self.nodeGenes), inNode, originalWeight, True)
                self.mutationSuccess = True
                print('added hidden node')
            else:
                print('Error! No connection found.')
        else:
            print('Could not find node(s). Did not mutate.')
    
    def mutateAddRandomNode(self):
        if (not self.connectGenes): # need to debug this 
            print('connectGenes is empty. can\'t mutateAddRandomNode')
            return
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            randomTuple = random.choice(list(self.connectGenes))
            self.mutateAddNode(randomTuple[0],randomTuple[1])
        self.mutationSuccess = False
    
    # this is so fucking slow
    def evaluateAction(self, observation):
        # TODO: IMPLEMENT THIS
        if not self.connectGenes:
            return env.action_space.sample()
        else:
            self.clearNodeValues()
            self.nodeGenes[1][1] = 1.0
            for i in range(self.inputCount-1):
                # print('Loading value: {} into node input: {}'.format(observation[i], i+2))
                self.nodeGenes[i+2][1] = observation[i]
            
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'hidden': # for every hidden node
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True: # if inNode == node
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = steepsigmoid(np.dot(x, w))
                    # print('hidden node #{}\'s value: {}'.format(node, self.nodeGenes[node][1]))
            softmaxsum = 0
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True:
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = np.exp(np.dot(x, w))
                    softmaxsum += np.exp(np.dot(x, w))
                    # print('output node #{}\'s unweighted value: {}'.format(node, self.nodeGenes[node][1]))
            output = []
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    output.append(self.nodeGenes[node][1] / softmaxsum)
            # print(output)
            # print('Genome takes action {}'.format(output.index(max(output))))
            return output.index(max(output))
        
    def evaluateActionBox(self, observation):
        # TODO: IMPLEMENT THIS
        if not self.connectGenes:
            return env.action_space.sample()
        else:
            self.clearNodeValues()
            self.nodeGenes[1][1] = 1.0
            for i in range(self.inputCount-1):
                # print('Loading value: {} into node input: {}'.format(observation[i], i+2))
                self.nodeGenes[i+2][1] = observation[i]
            
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'hidden': # for every hidden node
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True: # if inNode == node
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = steepsigmoid(np.dot(x, w))
                    # print('hidden node #{}\'s value: {}'.format(node, self.nodeGenes[node][1]))
            softmaxsum = 0
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True:
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = np.exp(np.dot(x, w))
                    softmaxsum += np.exp(np.dot(x, w))
                    # print('output node #{}\'s unweighted value: {}'.format(node, self.nodeGenes[node][1]))
            output = []
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    output.append(self.nodeGenes[node][1] / softmaxsum)
            # print(output)
            # print('Genome takes action {}'.format(output.index(max(output))))
            return output
    
    def evaluateSharedFitness(self, speciesCount):
        self.sharedFitness = self.fitness / speciesCount
    
    def printNodeGenes(self):
        print(self.nodeGenes)
        
    def printNodeGeneCount(self):
        print(len(self.nodeGenes))
        
    def printConnectGenes(self):
        print(self.connectGenes)
    
    def printConnectGeneCount(self):
        print(len(self.connectGenes))
    
    def printActiveConnectGenes(self):
        for connection in self.connectGenes:
            if (self.connectGenes[connection][2]):
                print(connection, self.connectGenes[connection][1])
        
    def printGlobalInnovation(self):
        print(self.globalInnovation)

> connection weights mutate as in any NE system, with each connection either perturbed or not at each generation 

In [ ]:
max_generations = 100
population_limit = 300
championThresholdCount = 5


species = {}
pop = {}

# NOTE: input/output changes depending on environment !
inputs = env.observation_space.shape[0] + 1
outputs = env.action_space.n


targetFitness = 1000 # this is for Bipedal-v3
maxTestTrials = 100
targetAverageFitness = 1000 # this is for Bipedal-v3
targetPosition = 0.5 # MountainCar-v0

solved = False

globalLoopCounter = 0
bestGenome = None
highestFitness = float("-inf")
window = sg.Window('Simulation Info', layout, finalize=True)
start_time = time.time()
for generation in range(max_generations):
    print('Beginning Generation #{}'.format(generation+1))
    window['GENER-NUM'].update(generation+1)
    if (generation == 0 and not pop): # start out with a uniform population of networks with zero hidden networks
        for genome in range(population_limit):
            g = Genome(inputs, outputs)
            Genome.resetGlobalInnovationNumber()
            g.initRandomGenome()
            pop[genome+1] = g
    
    # in each generation, genomes are sequentially placed into species 
    
    fitness = []
    
    for genome in pop:
        print('Beginning evaluation on genome #{}'.format(genome))
        window['GENOM-NUM'].update(genome)
        if (not species):
            print('creating new species:{}'.format(len(species)+1))
            species[len(species)+1] = {'summedSharedFitness': 0.0, 'maxFitness':0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter' : 0, 'isStagnant':False,'list':[pop[genome]], 'bestGenome': pop[genome]}
            pop[genome].species = len(species)
            print(len(species))
        else:
            for og in species:
                if (Genome.delta(pop[genome], species[og]['list'][0], c1, c2, c3) < deltaThreshold): # just compare with the first species
                    print('inserting into species#{}'.format(og))
                    species[og]['list'].append(pop[genome])
                    pop[genome].species = og
                    break
                else:
                    if (og == len(species)):
                        print('creating new species:{}'.format(len(species)+1))
                        species[len(species)+1] ={'summedSharedFitness': 0.0, 'maxFitness':0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter': 0, 'isStagnant':False,'list':[pop[genome]], 'bestGenome': pop[genome]}
                        pop[genome].species = len(species)
                        print(len(species))
                        break
        
        window['SPECIES-NUM'].update(pop[genome].species)
        observation = env.reset()
        pop[genome].resetFitness()
        # evalute the genome's performance # SDHJKLSDHLSJKDHLSDHJKLHDSKLDHJ
        t = 0
        globalLoopCounter += 1
        while(True):
            t+=1
            env.render()
            action = pop[genome].evaluateAction(observation)
            observation, reward, done, info = env.step(action)
            pop[genome].fitness += reward
            window['FITNESS-NUM'].update(pop[genome].fitness)
            window.finalize()
            if done:
                print("Episode finished after {} timesteps".format(t+1))
                print("Genome fitness: {}".format(pop[genome].fitness))
                fitness.append(pop[genome].fitness)
                if pop[genome].fitness > species[pop[genome].species]['maxFitness']:
                    species[pop[genome].species]['maxFitness'] = pop[genome].fitness
                    species[pop[genome].species]['stagnantCounter'] = 0
                    species[pop[genome].species]['isStagnant'] = False
                if pop[genome].fitness > highestFitness:
                    bestGenome = pop[genome]
                    highestFitness = pop[genome].fitness
                if pop[genome].fitness >= targetFitness: # changes depending on environment !
                    print('Found an optimal genome. testing it...')
                    testBenchTotalFitness = 0
                    for i in range(maxTestTrials):
                        m = 0
                        pop[genome].resetFitness()
                        observation = env.reset()
                        while(True):
                            m+=1
                            env.render()
                            action = pop[genome].evaluateAction(observation)
                            observation, reward, done, info = env.step(action)
                            pop[genome].fitness += reward
                            window['FITNESS-NUM'].update(pop[genome].fitness)
                            window.finalize()
                            if done:
                                testBenchTotalFitness+=pop[genome].fitness
                                print("Genome fitness: {}".format(pop[genome].fitness))
                                break
                    print('average fitness across 100 trials:{}'.format(testBenchTotalFitness/maxTestTrials))
                    if (testBenchTotalFitness/maxTestTrials >= targetAverageFitness):
                        print('found solution genome')
                        solved = True
                break
                
    if (solved):
        break
    
    
    
    print('Number of species at generation {}: {}'.format(generation+1, len(species)))
    # print('highest achieved score in this generation {}'.format(max(fitness)))
     
    # print(species[1])
    adjustedFitness = []
    totalAdjustedFitness = 0.0
    for genome in pop:
        pop[genome].evaluateSharedFitness(len(species[pop[genome].species]))
        adjustedFitness.append(pop[genome].sharedFitness)
        species[pop[genome].species]['summedSharedFitness'] += pop[genome].sharedFitness
        if pop[genome].sharedFitness > species[pop[genome].species]['maxAdjustedFitness']:
            species[pop[genome].species]['maxAdjustedFitness'] = pop[genome].sharedFitness
            species[pop[genome].species]['bestGenome'] = pop[genome]
            
    for s in species:
        totalAdjustedFitness += species[s]['summedSharedFitness']
        print('Maximum adjusted fitness in species {}: {}'.format(s, species[s]['maxAdjustedFitness']))
        print('Summed adjusted fitness in species {}: {}'.format(s, species[s]['summedSharedFitness']))
    
    # print('Total Adjusted Fitness across all species: {}'.format(totalAdjustedFitness))
    
    newPop = {}
    
    # the champion of each species with more than 5 networks will be copied into the next generation unchanged
    i = 0
    for s in species:
        if len(species[s]['list']) > championThresholdCount:
            newPop[i+1] = Genome.copyGenome(species[s]['bestGenome'])
            i += 1
    
    print('New Pop count after copying champions only: {}'.format(len(newPop)))
    print('i\'s value after inserting champions: {}'.format(i))
            
    
    # getting the proportion of offspring they are allowed
    totalBreedCount = 0
    for s in species:
        if species[s]['isStagnant']:
            print('species #{} is stagnant'.format(s))
            species[s]['breedAmountAllowed'] = 0
        else:
            species[s]['breedAmountAllowed'] = np.floor((species[s]['summedSharedFitness']/totalAdjustedFitness)*(population_limit-i)).astype(int)
            totalBreedCount += species[s]['breedAmountAllowed']

    
    leftovers = population_limit - i - totalBreedCount
    print('leftovers:{}'.format(leftovers))
        
    # get rid of the "lowest performing members" of each species (TODO)
    for s in species:
        if (len(species[s]['list']) > 1): # only get rid of the bottom 75 percent of each species
            amountToCull = np.floor((.75)*len(species[s]['list'])).astype(int)
            print('Eliminating {} genomes from species#{}'.format(amountToCull, s))
            # sort from ascending to descending 
            species[s]['list'].sort(reverse = False, key=lambda genome: genome.fitness)
            for q in range(amountToCull):
                species[s]['list'].pop()
    
    
    # now for each species breed !
    for s in species:
        print('species {} is allowed to have {} offspring'.format(s, species[s]['breedAmountAllowed']))
        for j in range(species[s]['breedAmountAllowed']):
            print('making offspring #{}'.format(i+1))
            if (random.random() <= mutateNoCrossoverChance or len(species[s]['list'])==1): # either the genome is a crossover or just the same genome 
                print('Chose no crossover')
                g = Genome.copyGenome(random.choice(species[s]['list']))
            else:
                print('Chose crossover')
                g1 = random.choice(species[s]['list'])
                g2 = random.choice(species[s]['list'])
                g = Genome.crossover(g1, g2)
            
            #print('Printing node genes:')
            #g.printNodeGenes()
            #print('Printing connect genes:')
            #g.printConnectGenes()
            
            # add a chance to mutate the connections
            if (g.connectGenes):
                if (random.random() <= genomeMutationChance):
                    print('mutation genome connections !')
                    for connection in g.connectGenes:
                        if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                            g.connectGenes[connection][1] += random.uniform(-perturbVal, perturbVal)
                        else:
                            g.connectGenes[connection][1] = random.uniform(-1.0, 1.0) # set a new value for 
            
                
            # chance to add new connections/nodes
            if (random.random() <= largePopulationNewNodeChance):
                print('trying to add new node')
                g.mutateAddRandomNode()
            if (random.random() <= largePopulationNewConnectionChance):
                print('trying to add new connection')
                g.mutateAddRandomConnection()
                
            newPop[i+1] = g
            i+=1
            
    # -----------------------------------------------------------------------------------------------------        
    
    if (leftovers > 0): 
        print('now for the leftovers:')
    for k in range(leftovers): # breeding the leftover genomes that were not calculated through floor
        if (random.random() <= interspeciesMatingRate): # very small chance
            print('interspecies')
            speciesKey1 = random.choice(list(species))
            speciesKey2 = random.choice(list(species))
            g1 = random.choice(species[speciesKey1]['list'])
            g2 = random.choice(species[speciesKey2]['list'])
            g = Genome.crossover(g1, g2)
            if (g.connectGenes):
                if (random.random() <= genomeMutationChance):
                    print('mutation genome connections !')
                    for connection in g.connectGenes:
                        if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                            g.connectGenes[connection][1] += random.uniform(-perturbVal, perturbVal)
                        else:
                            g.connectGenes[connection][1] = random.uniform(-1.0, 1.0) # set a new value for 
            
                
            # chance to add new connections/nodes
            if (random.random() <= largePopulationNewNodeChance):
                print('trying to add new node')
                g.mutateAddRandomNode()
            if (random.random() <= largePopulationNewConnectionChance):
                print('trying to add new connection')
                g.mutateAddRandomConnection()
                
                
            newPop[i+1] = g
            i+=1
        else: # else just find a random species and mate from there 
            print('random species mating')
            speciesKey = random.choice(list(species))
            if (random.random() <= mutateNoCrossoverChance or len(species[speciesKey]['list']) < 2): # either the genome is a crossover or just the same genome 
                print('Chose no crossover')
                g = Genome.copyGenome(random.choice(species[speciesKey]['list']))
            else:
                print('Chose crossover')
                g1 = random.choice(species[speciesKey]['list'])
                g2 = random.choice(species[speciesKey]['list'])
                g = Genome.crossover(g1, g2)
            
            #print('Printing node genes:')
            #g.printNodeGenes()
            #print('Printing connect genes:')
            #g.printConnectGenes()
            
            # add a chance to mutate the connections
            if (g.connectGenes):
                if (random.random() <= genomeMutationChance):
                    print('mutation genome connections !')
                    for connection in g.connectGenes:
                        if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                            g.connectGenes[connection][1] += random.uniform(-perturbVal, perturbVal)
                        else:
                            g.connectGenes[connection][1] = random.uniform(-1.0, 1.0) # set a new value for 
            
                
            # chance to add new connections/nodes
            if (random.random() <= largePopulationNewNodeChance):
                print('trying to add new node')
                g.mutateAddRandomNode()
            if (random.random() <= largePopulationNewConnectionChance):
                print('trying to add new connection')
                g.mutateAddRandomConnection()
                
                
            newPop[i+1] = g
            i+=1
            
    
    print('Final value of i: {}'.format(i))
    
    # ------------------------------------------------------------------------------------------
        
    # each existing species is represented by a random genome inside the species from the previous generation 
    if (generation >= 0):
        for s in species:
            species[s] = {'summedSharedFitness': 0.0, 'maxFitness': 0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter': species[s]['stagnantCounter']+1, 'isStagnant': False, 'list' : [random.choice(species[s]['list'])]}
            species[s]['bestGenome'] = species[s]['list'][0]
            if species[s]['stagnantCounter'] >= numUntilStagnant:  # if the maximum fitness of a species did not improve in 15 generations, the networks in the stagnant species were not allowed to reproduce 
                species[s]['isStagnant'] = True
    
    
    
    
    pop.clear()
    pop = newPop.copy()
print("--- %s seconds ---" % (time.time() - start_time))
print('Total number of evaluations: {}'.format(globalLoopCounter))
window.close()
env.close()
# saving contents of population into pickle when it is finished 
with open('data/pop.dictionary', 'wb') as config_dictionary_file:
    pickle.dump(pop, config_dictionary_file)

Beginning Generation #1
Beginning evaluation on genome #1
creating new species:1
1


C:\Users\valen\anaconda3\lib\site-packages\ipykernel_launcher.py:250: RuntimeWarning: overflow encountered in exp
C:\Users\valen\anaconda3\lib\site-packages\ipykernel_launcher.py:251: RuntimeWarning: overflow encountered in exp
C:\Users\valen\anaconda3\lib\site-packages\ipykernel_launcher.py:256: RuntimeWarning: invalid value encountered in double_scalars


Episode finished after 614 timesteps
Genome fitness: 110.0
Beginning evaluation on genome #2
inserting into species#1
Episode finished after 640 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #3
inserting into species#1
Episode finished after 680 timesteps
Genome fitness: 120.0
Beginning evaluation on genome #4
inserting into species#1
Episode finished after 523 timesteps
Genome fitness: 150.0
Beginning evaluation on genome #5
inserting into species#1
Episode finished after 647 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #6
inserting into species#1
Episode finished after 635 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #7
inserting into species#1
Episode finished after 646 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #8
inserting into species#1
Episode finished after 654 timesteps
Genome fitness: 110.0
Beginning evaluation on genome #9
inserting into species#1
Episode finished after 757 timesteps
Genome fitness: 240.0
B

Episode finished after 493 timesteps
Genome fitness: 160.0
Beginning evaluation on genome #72
inserting into species#1
Episode finished after 631 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #73
inserting into species#1
Episode finished after 644 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #74
inserting into species#1
Episode finished after 649 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #75
inserting into species#1
Episode finished after 650 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #76
inserting into species#1
Episode finished after 633 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #77
inserting into species#1
Episode finished after 647 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #78
inserting into species#1
Episode finished after 644 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #79
inserting into species#1
Episode finished after 651 timesteps
Genome fitness: 60

Episode finished after 646 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #141
inserting into species#1
Episode finished after 647 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #142
inserting into species#1
Episode finished after 644 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #143
inserting into species#1
Episode finished after 646 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #144
inserting into species#1
Episode finished after 645 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #145
inserting into species#1
Episode finished after 655 timesteps
Genome fitness: 200.0
Beginning evaluation on genome #146
inserting into species#1
Episode finished after 632 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #147
inserting into species#1
Episode finished after 641 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #148
inserting into species#1
Episode finished after 651 timesteps
Genome fit

...

Genome fitness: 270.0
Genome fitness: 320.0
Genome fitness: 530.0
Genome fitness: 410.0
Genome fitness: 1140.0
Genome fitness: 650.0
Genome fitness: 410.0
Genome fitness: 340.0
Genome fitness: 520.0
Genome fitness: 480.0
Genome fitness: 640.0
average fitness across 100 trials:519.0
Beginning evaluation on genome #2
inserting into species#1
Episode finished after 564 timesteps
Genome fitness: 260.0
Beginning evaluation on genome #3
inserting into species#1
Episode finished after 447 timesteps
Genome fitness: 110.0
Beginning evaluation on genome #4
inserting into species#1
Episode finished after 645 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #5
inserting into species#1
Episode finished after 645 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #6
inserting into species#1
Episode finished after 636 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #7
inserting into species#1
Episode finished after 653 timesteps
Genome fitness: 60.0
Beginning eva

Episode finished after 644 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #70
inserting into species#1
Episode finished after 644 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #71
inserting into species#1
Episode finished after 641 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #72
inserting into species#1
Episode finished after 557 timesteps
Genome fitness: 170.0
Beginning evaluation on genome #73
inserting into species#1
Episode finished after 656 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #74
inserting into species#1
Episode finished after 647 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #75
inserting into species#1
Episode finished after 635 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #76
inserting into species#1
Episode finished after 653 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #77
inserting into species#1
Episode finished after 641 timesteps
Genome fitness: 60

Episode finished after 636 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #140
inserting into species#1
Episode finished after 643 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #141
inserting into species#1
Episode finished after 636 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #142
inserting into species#1
Episode finished after 647 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #143
inserting into species#1
Episode finished after 647 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #144
inserting into species#1
Episode finished after 642 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #145
inserting into species#1
Episode finished after 656 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #146
inserting into species#1
Episode finished after 641 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #147
inserting into species#1
Episode finished after 639 timesteps
Genome fitn

Episode finished after 650 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #209
inserting into species#1
Episode finished after 662 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #210
inserting into species#1
Episode finished after 647 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #211
inserting into species#1
Episode finished after 644 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #212
inserting into species#1
Episode finished after 645 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #213
inserting into species#1
Episode finished after 633 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #214
inserting into species#1
Episode finished after 635 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #215
inserting into species#1
Episode finished after 640 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #216
inserting into species#1
Episode finished after 649 timesteps
Genome fitn

Episode finished after 641 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #278
inserting into species#1
Episode finished after 644 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #279
inserting into species#1
Episode finished after 655 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #280
inserting into species#1
Episode finished after 642 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #281
inserting into species#1
Episode finished after 658 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #282
inserting into species#1
Episode finished after 645 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #283
inserting into species#1
Episode finished after 637 timesteps
Genome fitness: 60.0
Beginning evaluation on genome #284
inserting into species#1


In [ ]:
testGenome = Genome.crossover(pop[1], pop[2])

In [ ]:
print(testGenome.connectGenes)

In [ ]:
print(bestGenome.nodeGenes)

In [ ]:
# test bench 

env = gym.make(envName)

testBenchTotalFitness = 0

for i in range(100):
    t = 0
    bestGenome.resetFitness()
    observation = env.reset()
    while(True):
        t+=1
        env.render()
        action = bestGenome.evaluateAction(observation)
        observation, reward, done, info = env.step(action)
        bestGenome.fitness += reward
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            testBenchTotalFitness+=bestGenome.fitness
            print("Genome fitness: {}".format(bestGenome.fitness))
            break

print('average fitness across 100 trials:{}'.format(testBenchTotalFitness/100))
env.close()